In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os
import enum

os.environ["CDF_LIB"] = "C:\cdf3.8.0_64bit_VS2015\lib"
from spacepy import pycdf
from pathlib import Path
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import datetime as dt

In [2]:
# Read hdf files. 
# file name should contain complete path to the file
def readH5File(fileName):
    df = pd.DataFrame()
    try:
        df = pd.read_hdf(fileName)
        return df
    except:
        error = 'Can not read file: ' + fileName 
        log(error)
    return df
        
# Read txt files. 
# file name should contain complete path to the file       
def readTextFile(fileName):
    try:
        dataFile = open(fileName, 'r')
        txtData = dataFile.read()
        return txtData
    except:
        error = 'Can not read file: ' + fileName 
        log(error)
    
    
# Read cdf (common data format). Refer https://cdf.gsfc.nasa.gov/
# Make sure cdf library is installed and CDF_LIB path is defined
# Link : https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/cdf38_0/windows/
# os.environ["CDF_LIB"] = "C:\cdf3.8.0_64bit_VS2015\lib"
# Make sure pycdf libray is installed
def readCDFFile(fileName):
    try:
        cdfFile = pycdf.CDF(fileName)
        return cdfFile
    except:
        error = 'Can not read file: ' + fileName 
        log(error)
        
def saveToHDFFile(dataframe, filePath):
    try:
        dataframe.to_hdf(filePath, key='df')
    except:
        error = 'Failed to save dataframe: ' + filePath 
        log(error)

In [3]:
# calculates number of days between given dates
# this is used verify there are accurate number of entries in the data set
def numberOfDaysInDates(start, end):
    startDate = datetime.strptime(start, dateFormat)
    endDate = datetime.strptime(end, dateFormat)
    delta = endDate - startDate
    return delta.days + 1


# An empty dataframe is initialised with the given metadata
# attribute name in metadata is used to create the column names
def initDataFrame(metadata):
    columnNames = []
    for element in metadata:
        attribute = element[_attribute] if _attribute in element else ''
        columnNames.append(attribute)   
        
    df = pd.DataFrame(columns=columnNames)
    return df


# extract data values from text lines based on the given meta data
# metadata comprises what is the data, type of the data and positions need to be extracted
# Threshold value: if the attribute value goes beyond thresold value then it is considered as NaN
def extractDataFromTxtLine(txt, metadata):
    value = {}
    for element in metadata:
        attribute = element[_attribute] if _attribute in element else ''
        startPosition = element[_start_position] if _start_position in element else 0
        endPosition = element[_end_position] if _end_position in element else 0
        thresholdValue = element[_threshold_value] if _threshold_value in element else 0
        dataDateFormat = element[_date_format] if _date_format in element else None
        try:
            attrValue = txt[startPosition:endPosition]
            if attrValue == '':
                break
            if thresholdValue != 0:
                try:
                    attrValue = float(attrValue)
                    attrValue = float('NaN') if attrValue <= 0 else attrValue
                    # checks attribute value goes above threshold value. if goes beyod invalidate
                    if attrValue > thresholdValue:
                        attrValue = float('NaN')
                except:
                    attrValue = float('NaN')
                    error = 'Value cannot be converted to float'
                    log(error)
            value[attribute] = attrValue
        except:
            error = 'No data available at this position'
            log(error)
    return value


# make date formats in data to yyyy-mm-dd
# remove redundant datas
# add values to time series from start date to end date if any entries are missing
def cleanAndFormatData(data, metadata):
    dataDateFormat = None
    for element in metadata:
        dataDateFormat = element[_date_format] if _date_format in element else None
    # set date format to yyyy-mm-dd
    data[_date] = pd.to_datetime(data[_date], format=dataDateFormat)
    
    # extract data only inbetween start date and end date(analysis time period)
    data = data[(data[_date] >= startDate) & (data[_date] <= endDate)]
    
    
    # set date as index for the dataframe
    data = data.set_index(_date)
    
    # remove duplicates
    data = data[~data.index.duplicated(keep='first')]
    
    #check final data set has accurate number of entries
    dataLength = data.shape[0]
    if dataLength != numberOfDays:
        error = 'Number of entries mismatch. Check data'
        log(error)
        log('Automatic index correction')
        data = fillTimeSeries(data)
    return data


# fill time series index if any dates are missing
# index is filled by a start date and end date. 
# It will check for any date missing between start and end date.
# Creates series from start and end date then reindex the original data set with the date series.
# fills NaN for missing values
def fillTimeSeries(data):
    strSDate = datetime.strptime(startDate, '%Y-%m-%d')
    strEDate = datetime.strptime(endDate, '%Y-%m-%d')
    timeSeries = pd.DataFrame({ _date: pd.Series([strSDate, strEDate])})
    timeSeries.set_index(_date, inplace=True)
    data = data.append(timeSeries)
    idx = pd.date_range(min(data.index), max(data.index))
    data = data[~data.index.duplicated(keep='first')]
    data = data.reindex(idx, fill_value=float('NaN'))
    return data


# Fills missing values by interpolation. 
def fillNaNValueWithInterpolation(data, column):
    data[column] = data[column].interpolate()
    data[column].head(4)
    
    return data


# calculate moving average with a window
# window size is 27 and minimum period is 17
def calculateMovingAvg(data, valueColumn, movingAvgColumn):
    data[movingAvgColumn] = data[valueColumn].rolling(window=windowSize, min_periods=windowSize-10).mean()
    return data


# calculate relative difference of original value and moving average value
def calculateRelativeDifference(data, valueColumn, movingAvgColumn, relDiffColumn):
    data[relDiffColumn] = ((data[valueColumn] - data[movingAvgColumn])/data[movingAvgColumn])*100
    return data

def getStartAndEndDateOfYear(year):
    first_day_of_year = str(year) + '-01-01'
    last_day_of_year = str(year) + '-12-31'
    
    return (first_day_of_year, last_day_of_year)


In [4]:
startYear = 1997
endYear = 2020
startDate = str(startYear) + '-01-01'
endDate = str(endYear) + '-12-31'
analysisPeriod = np.arange(startYear, endYear+1)
dateFormat = '%Y-%m-%d'
numberOfDays = numberOfDaysInDates(startDate, endDate)

f10_7_max_value = 300
sws_max_value = 900

windowSize = 27
minPeriod = 20

_attribute = 'attribute'
_start_position = 'start_position'
_end_position = 'end_position'
_date = 'date'
_threshold_value = 'threshold_value'
_date_format = 'date_format'

# ------------------f10.7 variables
_f10_7 = 'f10_7'
_f10_7_ma_27 = 'f10_7_ma_27'
_f10_7_rel_diff = 'f10_7_diff_rel'

# ------------------solar wind speed variables
_sws = 'sws'
_sws_ma_27 = 'sws_ma_27'
_sws_rel_diff = 'sws_diff_rel'

#-------------------total electron content variables


final_plot_location = 'C:/Users/davi_fr/Documents/Thesis Project Final/Final Plot/'

In [8]:
F10_7_FOLDER = "C:/Users/davi_fr/Documents/Project/data/F10.7cm radiofluxindex/"

# dataPosition variable contain how to extarct data from string based on position and length
# format: [[attribure, start_position, end_position, threshold_value, date_format],...]
# threshold value: 
f10_7DataPosition = [{_attribute: _date, _start_position: 0,  _end_position: 10},
                {_attribute: _f10_7, 
                 _start_position: 149, 
                 _end_position: 156, 
                 _threshold_value: f10_7_max_value, 
                 _date_format: None}]




f10_7_file_path = 'C:/Users/davi_fr/Documents/Thesis Project Final/data/f10_7/f10_7.h5'
sws_file_path = 'C:/Users/davi_fr/Documents/Thesis Project Final/data/sws/sws.h5'

In [7]:
def writeToLogFile(log):
    dt = datetime.now()
    log = str(dt) + '\t' + log + '\n' 
    file_name = 'log_file.txt'
    f = open(file_name, 'a+')  # open file in append mode
    f.write(log)
    f.close()

def log(message):
    writeToLogFile(message)
#     print(message)

In [6]:
testLatitudes = np.arange(-87.5, 90, 2.5)
print(testLatitudes)

[-87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5 -60.
 -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5 -30.
 -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5   0.
   2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5  30.
  32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5  60.
  62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5]


In [5]:
tec_rel_diff = "C:\\Users\\davi_fr\\Documents\\Thesis Project Final\\tets folder main\\test\\computations\\relative_difference\\tec"
tec_file = "C:\\Users\\davi_fr\\Documents\\Thesis Project Final\\tets folder main\\test\\data_formatting\\tec_og"

tecRelDiffColumnName = "tec_rel_diff_lat_"
tecColumnName = "tec_lat_"

In [9]:


def getTECDataAtLatitude(latitude):
    tec_data_values = pd.DataFrame()
    for year in analysisPeriod:
        tec_file_path = tec_rel_diff + "\\" + str(year) + ".h5"
        tec_og_file_path = tec_file + "\\" + str(year) + ".h5"

        tecData = readH5File(tec_file_path)
        tecOGData = readH5File(tec_og_file_path)

        tecData = pd.DataFrame(tecData[[latitude]].mean(axis=1).groupby(level='date').mean())
        tecOGData = pd.DataFrame(tecOGData[[latitude]].mean(axis=1).groupby(level='date').mean())


        tecData.columns = [tecRelDiffColumnName+str(latitude)]
        tecData[tecColumnName+str(latitude)] = tecOGData
        tecData.index =  pd.to_datetime(tecData.index, format='%Y-%m-%d')
        tec_data_values = tec_data_values.append(tecData)
    return tec_data_values

# tec_data = getTECDataAtLatitude(0)    
# print(tec_data)

In [10]:
f10_7_rel_diff_file = "C:\\Users\\davi_fr\\Documents\\Thesis Project Final\\tets folder main\\test\\computations\\relative_difference\\f10_7\\f10_7.h5"
f10_7_rel_diff = readH5File(f10_7_rel_diff_file)

f10_7_file = "C:\\Users\\davi_fr\\Documents\\Thesis Project Final\\tets folder main\\test\\data_formatting\\f10_7\\f10_7.h5"
f10_7Data = readH5File(f10_7_file)
f10_7Data['f10_7_rel_diff'] = f10_7_rel_diff['f10_7']
print(f10_7Data)

sws_rel_diff_file = "C:\\Users\\davi_fr\\Documents\\Thesis Project Final\\tets folder main\\test\\computations\\relative_difference\\sws\\sws.h5"
sws_rel_diff = readH5File(sws_rel_diff_file)


sws_file = "C:\\Users\\davi_fr\\Documents\\Thesis Project Final\\tets folder main\\test\\data_formatting\\solar_wind_speed\\solar_wind_speed.h5"
swsData = readH5File(sws_file)
swsData['sws_rel_diff'] = sws_rel_diff['sws']
print(swsData)


            f10_7  f10_7_rel_diff
date                             
1997-01-01   70.0             NaN
1997-01-02   69.7             NaN
1997-01-03   70.8             NaN
1997-01-04   71.4             NaN
1997-01-05   71.9             NaN
...           ...             ...
2020-12-27   84.9        0.258048
2020-12-28   84.3        0.290813
2020-12-29   81.4       -2.276567
2020-12-30   80.1       -2.983133
2020-12-31   78.5       -4.298551

[8766 rows x 2 columns]
               sws  sws_rel_diff
1997-01-01     NaN           NaN
1997-01-02     NaN           NaN
1997-01-03     NaN           NaN
1997-01-04     NaN           NaN
1997-01-05     NaN           NaN
...            ...           ...
2020-12-27  463.09     17.168871
2020-12-28  486.17     22.138036
2020-12-29  467.50     16.778766
2020-12-30  478.41     18.260138
2020-12-31  398.60     -1.849074

[8766 rows x 2 columns]


In [11]:
def calculateCorrelationWithWindow(latitude, tec_data_file, f10_7Data, swsData):
    correlation_window_file = pd.DataFrame(columns=['window_size', 'correlation_tec_f10_7_og', 'correlation_f10_7_rel_diff', 'correlation_sws_rel_diff'])
    for windowSize in np.arange(1, 8766):
        
        correlation_tec_f10_7_og = tec_data_file[tecColumnName+str(latitude)].rolling(windowSize).corr(f10_7Data['f10_7']).mean()
        correlation_f10_7_rel_diff = tec_data_file[tecRelDiffColumnName+str(latitude)].rolling(windowSize).corr(f10_7Data['f10_7_rel_diff']).mean()
        correlation_sws_rel_diff = tec_data_file[tecRelDiffColumnName+str(latitude)].rolling(windowSize).corr(swsData['sws_rel_diff']).mean()
        correlation_sws_og = tec_data_file[tecColumnName+str(latitude)].rolling(windowSize).corr(swsData['sws']).mean()

        row = {'window_size': windowSize, 
               'correlation_tec_f10_7_og': correlation_tec_f10_7_og, 
               'correlation_f10_7_rel_diff': correlation_f10_7_rel_diff, 
               'correlation_sws_rel_diff':correlation_sws_rel_diff,
               'correlation_sws_og':correlation_sws_og
              }
        correlation_window_file = correlation_window_file.append(row, ignore_index=True)
#     print(correlation_window)  

    correlation_window_file.set_index('window_size', inplace = True)
    fileName = "window_size_" + str(latitude) + ".h5"
    correlation_window_file.to_hdf("C:/Users/davi_fr/Documents/Thesis Project Final/source_code/correlation_window_size/" + fileName, key= 'df')
    return correlation_window_file


# correlation_window = calculateCorrelationWithWindow(0, tec_data, f10_7Data, swsData)

In [12]:
df = correlation_window[['correlation_tec_f10_7_og', 'correlation_f10_7_rel_diff', 'correlation_sws_rel_diff']]
# df = df[0:100]
ax = df.plot(figsize=(12, 5))
lgd = ax.legend(['Mean Correlation Coefficient; TEC X F10.7',
           'Mean Correlation Coefficient; Relative Difference TEC X F10.7',
           'Mean Correlation Coefficient; Relative Difference TEC X Solar Wind Speed',
           'Mean Correlation Coefficient; TEC X Solar Wind Speed'], 
          bbox_to_anchor= (1, -0.2))

plt.title('Mean correlation coefficient at 0 degree latitude X Correlation Window Size', pad=20)
ax.set_xlabel("Window Size")
ax.set_ylabel("Correlation Coefficient")
ax.grid('on', which='minor', axis='x' )
ax.grid('on', which='major', axis='x' )
ax.grid('on', which='minor', axis='y' )
ax.grid('on', which='major', axis='y' )

minorLocator = MultipleLocator(500)
# majorLocator = MultipleLocator(900)
ax.xaxis.set_minor_locator(minorLocator)
ax.xaxis.set_major_locator(minorLocator)
# correlation_window.max()

# df.to_excel("correlation_window_size.xlsx") 

plt.savefig('Mean correlation coefficient at 0 degree latitude X Correlation Window Size.jpg',
            bbox_extra_artists=(lgd,), 
            bbox_inches='tight', dpi= 200)

NameError: name 'correlation_window' is not defined

In [13]:
testLatitudes = np.arange(-87.5, 90, 2.5)
for latitude in testLatitudes:
    print('______________' + str(latitude) + '_______________')
    tec_data_file_values = getTECDataAtLatitude(latitude)
    print('TEC Data Fetched at ' + str(latitude))
    correlation_window_file_temp = calculateCorrelationWithWindow(latitude, tec_data_file_values, f10_7Data, swsData)
    print('Correlation Data saved at ' + str(latitude))
    
#     finalDataFrame = correlation_window
#     currentColumns = finalDataFrame.columns
#     latitudeColumn = [latitude]
#     finalDataFrame.columns = pd.MultiIndex.from_product([latitudeColumn, currentColumns], names=['latitude', 'data'])
    
print('end')
    
    

______________-10.0_______________
TEC Data Fetched at -10.0
Correlation Data saved at -10.0
______________-7.5_______________
TEC Data Fetched at -7.5
Correlation Data saved at -7.5
______________-5.0_______________
TEC Data Fetched at -5.0
Correlation Data saved at -5.0
______________-2.5_______________
TEC Data Fetched at -2.5
Correlation Data saved at -2.5
______________0.0_______________
TEC Data Fetched at 0.0
Correlation Data saved at 0.0
______________2.5_______________
TEC Data Fetched at 2.5
Correlation Data saved at 2.5
______________5.0_______________
TEC Data Fetched at 5.0
Correlation Data saved at 5.0
______________7.5_______________
TEC Data Fetched at 7.5
Correlation Data saved at 7.5
______________10.0_______________
TEC Data Fetched at 10.0
Correlation Data saved at 10.0
______________12.5_______________
TEC Data Fetched at 12.5
Correlation Data saved at 12.5
______________15.0_______________
TEC Data Fetched at 15.0
Correlation Data saved at 15.0
______________17.5_